# Avaliação automatizada de LLMs utilizando DeepEval - Faithfulness (Fidelidade)


In [1]:
import os
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from deepeval.test_case import LLMTestCase
from deepeval.metrics import FaithfulnessMetric

from IPython.display import display, Markdown

C:\Users\leona\Documents\GitHub\ollama_tests\.venv\Lib\site-packages\deepeval\__init__.py:49: UserWarning: You are using deepeval version 1.4.9, however version 1.5.0 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [2]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

## 1. Fidelidade

Alinhada factual e semanticamente com o conteúdo fornecido no contexto de recuperação. Em outras palavras, ela verifica se a resposta do modelo é fiel às informações presentes no contexto recuperado, garantindo que não haja contradições ou informações não suportadas pelo contexto fornecido.

Diferentemente da métrica de "Alucinação" (HallucinationMetric), que foca em identificar informações inventadas pelo modelo, a métrica de Fidedignidade concentra-se em detectar contradições entre a saída gerada e o contexto de recuperação.

**Alice’s Adventures in Wonderland** Chapter 1

Ref do livro: https://www.gutenberg.org/files/11/11-0.txt

In [3]:
livro_cp1 = ""
with open("./short_story.txt", "r", encoding="utf-8") as arquivo:
    livro_cp1 = arquivo.read()


display(Markdown(f'{livro_cp1[:492]}...\n\n...\n\n...{livro_cp1[-500:]}'))
print(len(livro_cp1))

CHAPTER I.
Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy...

...

...ate a little bit, and said anxiously to herself, “Which way? Which
way?”, holding her hand on the top of her head to feel which way it was
growing, and she was quite surprised to find that she remained the same
size: to be sure, this generally happens when one eats cake, but Alice
had got so much into the way of expecting nothing but out-of-the-way
things to happen, that it seemed quite dull and stupid for life to go
on in the common way.

So she set to work, and very soon finished off the cake.

11285


In [4]:
PROMPT = f"""
"Read the following excerpt from the book carefully. Pay attention to the details and try to imagine the scene as described. Once you’ve thoroughly understood the passage, consider any characters Alice might envision interacting with as she experiences this moment. After reading, answer the question thoughtfully and accurately based on the text.

Here is the text:

{{TEXT}}

Question: While falling down the rabbit hole, who does Alice imagine having a conversation with?"
"""

prompt = PROMPT.format(TEXT = livro_cp1)
display(Markdown(f'{prompt[:1000]}...'))


"Read the following excerpt from the book carefully. Pay attention to the details and try to imagine the scene as described. Once you’ve thoroughly understood the passage, consider any characters Alice might envision interacting with as she experiences this moment. After reading, answer the question thoughtfully and accurately based on the text.

Here is the text:

CHAPTER I.
Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

The...

In [5]:
chat = ChatOpenAI(model="gpt-4o-mini")
response = chat.invoke(prompt)

In [6]:
display(Markdown(f'{response.content}'))

While falling down the rabbit hole, Alice imagines having a conversation with her cat, Dinah. She expresses her thoughts about Dinah missing her and wonders if Dinah would enjoy being with her in this strange situation. Alice also muses about whether cats eat bats, reflecting her whimsical and curious nature as she tumbles through the air.

### Avaliação da resposta

ref.: https://www.sparknotes.com/lit/alice/section1/?quickquiz_id=1709

In [7]:
actual_output = response.content

# Replace this with the actual retrieved context from your RAG pipeline
retrieval_context = [livro_cp1]

metric = FaithfulnessMetric(
    model="gpt-4o-mini",
    include_reason=True
)
test_case = LLMTestCase(
    input="While falling down the rabbit hole, who does Alice imagine having a conversation with?",
    actual_output=actual_output,
    retrieval_context=retrieval_context
)

metric.measure(test_case)
print(metric.score)
print(metric.reason)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

0.8
The score is 0.80 because the actual output fails to capture Alice's imaginative aspect of having a conversation with her cat, Dinah, which was an important detail in the retrieval context.
